In [1]:
import predict
import cv2
import numpy
from matplotlib import pyplot as plt
import math
from map import Map
from grid import DiscreteProbabilityDistribution

2023-06-19 09:21:08.671340: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-19 09:21:09.346321: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-19 09:21:09.346412: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-19 09:21:09.346419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2023-06-19 09:21:10.867233: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/karpase/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-06-19 09:21:10.867314: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Load the NN model

In [2]:
model_name = "../resource/v7-LandCover-retrained-twice"
model = predict.load_model(model_name)

cover_distributions_filename = "../resource/cover_distribution.yaml"
cover_distributions = predict.load_cover_distributions(cover_distributions_filename)
print(cover_distributions)


{0: DiscreteProbabilityDistribution: {0: 1.0}, 1: DiscreteProbabilityDistribution: {0: 0.9, 1: 0.1}, 2: DiscreteProbabilityDistribution: {0: 0.8, 1: 0.1, 5: 0.1}, 3: DiscreteProbabilityDistribution: {0: 1.0}, 4: DiscreteProbabilityDistribution: {0: 0.7, 1: 0.1, 5: 0.1, 10: 0.1}}


## Load and Show the map

In [3]:

map = Map(0, 100, 0, 100, "kaka_input.tif")

polygons = [
    ((10, 10), (10,20), (40, 10)),
    ((35, 35), (45, 45), (35, 55), (25, 45)),
    ((60,60), (60, 90), (90, 90), (90, 60))
]

polygon_disributions = [
    DiscreteProbabilityDistribution({1: 0.8, 5: 0.2}),
    DiscreteProbabilityDistribution({5: 0.9, 10: 0.1}),
    DiscreteProbabilityDistribution({1: 0.3, 5: 0.2, 10: 0.5}),
]


map_grid_cell_size = (10, 10)

plt.title('Input Map')
plt.imshow(map.image)

## Predict the cover labels


In [ ]:
pred = predict.image_to_pixel_cover(model, map.image)
map_pred = Map(0, 100, 0, 100, pred)


## Show the cover labels


* black - Background
* red - building
* green - woodland
* blue - water
* white - road


In [ ]:
plt.title('Cover Labels')
plt.imshow(map_pred.colormap())


## Add the Polygons to the Cover Labels

Each polygon gets a unique color


In [ ]:
polygon_id = predict.NUM_CLASSES

for p, d in zip(polygons, polygon_disributions):     
    map_pred.color_polygon(p, polygon_id)   
    cover_distributions[polygon_id] = d
    polygon_id = polygon_id + 1
    
print(cover_distributions)

plt.imshow(map_pred.colormap())
plt.title('Map with Polygons')    



## Overlay a Grid


In [ ]:
grid = map_pred.to_grid(map_grid_cell_size[0], map_grid_cell_size[1], cover_distributions)


print(grid)
